In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp "/content/drive/MyDrive/Projet nlp/sanad_dataset.zip" /content/sanad_dataset.zip

cp: cannot stat '/content/drive/MyDrive/Projet nlp/sanad_dataset.zip': No such file or directory


In [3]:
# !ls -lh /content/

total 8.0K
drwx------ 6 root root 4.0K Feb 10 23:12 drive
drwxr-xr-x 1 root root 4.0K Feb  7 14:20 sample_data


In [7]:
# !pip install datasets
print("✔️ datasets est bien installé !")

✔️ datasets est bien installé !


In [8]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os
import pandas as pd
import numpy as np
from datasets import Dataset

In [9]:
# Vérifier si CUDA est disponible
DEVICE = "cuda" if torch.cuda.is_available() else -1
print(f"🔹 Utilisation de l'appareil : {DEVICE}")

# Charger le modèle de résumé
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").to(DEVICE)

# Pipeline de résumé
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer, device=0 if DEVICE == "cuda" else -1)


🔹 Utilisation de l'appareil : cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
df_clean = pd.read_csv("/content/drive/MyDrive/Projet nlp/sanad_dataset_clean.csv")

NameError: name 'pd' is not defined

In [ ]:
df_clean.head()

,filename,category,text
0,4705.txt,Culture,يبدو أن أزمة الحفل الموسيقي، الذي تدخلت نقابة ...
1,2497.txt,Culture,كشف المغني البريطاني من أصل لبناني ميكا عن مشر...
2,1124.txt,Culture,أكد الفنان الكويتي عبدالحسين عبدالرضا أنه يعتب...
3,0182.txt,Culture,دشنت مدينة نابلس، أمس الاثنين، أكبر جدارية تضم...
4,0681.txt,Culture,"أقرت المغنية البريطانية ""اديل"" بأنها بكت طويلا..."


In [ ]:
print(df_clean.shape)
print(f"🔹 Taille du DataFrame complet : {df_clean.shape}")

# Diviser le DataFrame en 10 chunks
chunks = np.array_split(df_clean, 20)
print(f"🔹 Taille du premier chunk : {chunks[0].shape}")

# Travailler uniquement avec le premier chunk
first_chunk = chunks[0]

# Convertir le premier chunk en Dataset Hugging Face
dataset = Dataset.from_pandas(first_chunk)

(205760, 3)
🔹 Taille du DataFrame complet : (205760, 3)
🔹 Taille du premier chunk : (10288, 3)


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)


Device set to use cuda:0


In [ ]:
# Fonction pour générer un résumé en batch
def summarize_batch(batch):
    # Limiter la longueur du texte à 1024 caractères
    truncated_texts = [text[:1024] for text in batch["text"]]
    summaries = summarization_pipeline(truncated_texts, max_length=100, min_length=20, do_sample=False)
    batch["summary"] = [s["summary_text"] for s in summaries]
    return batch


# Appliquer le résumé en batch
dataset = dataset.map(summarize_batch, batched=True, batch_size=4)

# Convertir le Dataset Hugging Face en DataFrame Pandas
df = dataset.to_pandas()

# Sauvegarder le chunk résumé
df.to_csv("/content/sanad_dataset_summarized_chunk_1.csv", index=False)
print("✔️ Résumés du premier chunk sauvegardés dans : /content/sanad_dataset_summarized_chunk_1.csv")

Map:   0%|          | 0/10288 [00:00<?, ? examples/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
